In [12]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

Load data and subtract water

In [13]:
data_path = Path('~/workspace/data/lipid/GIXD').expanduser()
plot_path = Path('~/workspace/plot/lipid/GIXD').expanduser()

In [14]:
def load_gixd_data(path: str, name: str, index: list, pressure: list):
    """
    Load GIXD data from specified paths.
    
    Args:
        path: Base directory path
        name: Base name for files
        index: List of index numbers
        pressure: List of pressure values
        
    Returns:
        Tuple containing:
        - I: List of intensity data arrays
        - Qxy: List of Qxy data arrays
        - Qz: List of Qz data arrays
    """
    
    # Create paths
    data_path = Path(path) / f'{name}'
    I_paths = [data_path / f'{name}_{i}_{i}_combined_I.dat' for i in index]
    Qxy_paths = [data_path / f'{name}_{i}_{i}_combined_Qxy.dat' for i in index]
    Qz_paths = [data_path / f'{name}_{i}_{i}_combined_Qz.dat' for i in index]
    
    # Load data
    I = [np.loadtxt(path) for path in I_paths]
    Qxy = [np.loadtxt(path) for path in Qxy_paths]
    Qz = [np.loadtxt(path) for path in Qz_paths]

    I = np.nan_to_num(I, nan=0)
    
    return I, Qxy, Qz

In [15]:
water_index = [44]
water_pressure = [0.4]
water_I, water_Qxy, water_Qz = load_gixd_data(data_path, 'water', water_index, water_pressure)

azotrans_index = [54, 58, 62]
azotrans_pressure = [10, 20, 30]
azotrans_I, azotrans_Qxy, azotrans_Qz = load_gixd_data(data_path, 'azotrans', azotrans_index, azotrans_pressure)
azotrans_I -= water_I[0]

azocis_index = [78, 82, 86, 90]
azocis_pressure = [5, 10, 20, 30]
azocis_I, azocis_Qxy, azocis_Qz = load_gixd_data(data_path, 'azocis', azocis_index, azocis_pressure)
azocis_I -= water_I[0]

azocis02_index = [106, 110]
azocis02_pressure = [3.3, 30]
azocis02_I, azocis02_Qxy, azocis02_Qz = load_gixd_data(data_path, 'azocis02', azocis02_index, azocis02_pressure)
azocis02_I -= water_I[0]

azocis03_index = [115, 119]
azocis03_pressure = [0.1, 30]
azocis03_I, azocis03_Qxy, azocis03_Qz = load_gixd_data(data_path, 'azocis03', azocis03_index, azocis03_pressure)
azocis03_I -= water_I[0]

dopc_index = [16, 12, 20, 24]
dopc_pressure = [0.1, 10, 20 ,30]
dopc_I, dopc_Qxy, dopc_Qz = load_gixd_data(data_path, 'dopc', dopc_index, dopc_pressure)
dopc_I -= water_I[0]

redazo_index = [128, 132, 136, 140]
redazo_pressure = [0.1, 10, 20, 30]
redazo_I, redazo_Qxy, redazo_Qz = load_gixd_data(data_path, 'redazo', redazo_index, redazo_pressure)
redazo_I -= water_I[0]

In [16]:
print("Qxy", water_Qxy[0].min(), water_Qxy[0].max(), water_Qxy[0][1]-water_Qxy[0][0])
print("Qz", water_Qz[0].min(), water_Qz[0].max(), water_Qz[0][1]-water_Qz[0][0])

Qxy 0.655 1.8575 0.0024999999999999467
Qz -0.0175 1.34 0.0025000000000000022


Cartesian to polar coordinates

In [17]:
from xrd_anlyutils.math import transform
from xrd_anlyutils.math import roi

In [18]:
def plot_gixd(I, Qxy, Qz, name, vmin=None, vmax=None, savepath=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    im1 = ax1.imshow(I, cmap='inferno', extent=[np.min(Qxy), np.max(Qxy), np.min(Qz), np.max(Qz)], origin='lower', aspect='auto', vmin=vmin, vmax=vmax)
    ax1.set_xlabel('Qxy [A$^{-1}$]')
    ax1.set_ylabel('Qz [A$^{-1}$]')
    ax1.set_title('Cartesian')
    plt.colorbar(im1, ax=ax1)

    I_polar, Q, theta = transform.cartesian2polar(I, Qxy, Qz, 0.01, 0.01)

    im2 = ax2.imshow(I_polar, cmap='inferno', extent=[np.min(Q), np.max(Q), np.rad2deg(np.min(theta)), np.rad2deg(np.max(theta))], origin='lower', aspect='auto', vmin=vmin, vmax=vmax)
    ax2.set_xlabel('Q [A$^{-1}$]')
    ax2.set_ylabel('theta [deg]')
    ax2.set_title('Polar Rebinning')
    plt.colorbar(im2, ax=ax2)

    fig.suptitle(name)
    if savepath is not None:
        if not savepath.exists():
            savepath.mkdir(parents=True, exist_ok=True)
        plt.savefig(savepath / f'{name}.png')
        plt.close(fig)
    else:
        plt.show()

In [19]:
for i in range(len(azotrans_I)):
    plot_gixd(azotrans_I[i], azotrans_Qxy[i], azotrans_Qz[i], f'Azotrans {azotrans_pressure[i]}', 0, 10, plot_path / "azotrans")
for i in range(len(azocis_I)):
    plot_gixd(azocis_I[i], azocis_Qxy[i], azocis_Qz[i], f'Azocis {azocis_pressure[i]}', 0, 10, plot_path / "azocis")
for i in range(len(azocis02_I)):
    plot_gixd(azocis02_I[i], azocis02_Qxy[i], azocis02_Qz[i], f'Azocis02 {azocis02_pressure[i]}', 0, 10, plot_path / "azocis02")
for i in range(len(azocis03_I)):
    plot_gixd(azocis03_I[i], azocis03_Qxy[i], azocis03_Qz[i], f'Azocis03 {azocis03_pressure[i]}', 0, 10, plot_path / "azocis03")
for i in range(len(dopc_I)):
    plot_gixd(dopc_I[i], dopc_Qxy[i], dopc_Qz[i], f'Dopc {dopc_pressure[i]}', 0, 10, plot_path / "dopc")
for i in range(len(redazo_I)):
    plot_gixd(redazo_I[i], redazo_Qxy[i], redazo_Qz[i], f'Redazo {redazo_pressure[i]}', 0, 10, plot_path / "redazo")

In [28]:
def plot_peak_profile(I, Qxy, Qz, name, savepath=None, labels=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    for i in range(len(I)):
        I_polar, Q, theta = transform.cartesian2polar(I[i], Qxy[i], Qz[i], 0.01, 0.01)
        ax1.plot(Q, i + roi.get_avg([0.7, 1.8, 0, 0.2], I_polar, Q, theta, axis='y'), label=f'{labels[i]}')
        ax2.plot(np.rad2deg(theta), 2 * i + roi.get_avg([1.25, 1.5, 0, 0.8], I_polar, Q, theta, axis='x'), label=f'{labels[i]}')
    ax1.legend()
    ax1.set_xlabel('Q [A$^{-1}$]')
    ax1.set_ylabel('Intensity [a.u.]')
    ax1.set_title(name)
    ax2.legend()
    ax2.set_xlabel('theta [deg]')
    ax2.set_ylabel('Intensity [a.u.]')
    ax2.set_title(name)
    if savepath is not None:
        if not savepath.exists():
            savepath.mkdir(parents=True, exist_ok=True)
        plt.savefig(savepath / f'{name}.png')
        plt.close(fig)
    else:
        plt.show()

In [29]:
plot_peak_profile(azotrans_I, azotrans_Qxy, azotrans_Qz, 'Azotrans', savepath=plot_path / "azotrans", labels=azotrans_pressure)
plot_peak_profile(azocis_I, azocis_Qxy, azocis_Qz, 'Azocis', savepath=plot_path / "azocis", labels=azocis_pressure)
plot_peak_profile(azocis02_I, azocis02_Qxy, azocis02_Qz, 'Azocis02', savepath=plot_path / "azocis02", labels=azocis02_pressure)
plot_peak_profile(azocis03_I, azocis03_Qxy, azocis03_Qz, 'Azocis03', savepath=plot_path / "azocis03", labels=azocis03_pressure)
plot_peak_profile(dopc_I, dopc_Qxy, dopc_Qz, 'Dopc', savepath=plot_path / "dopc", labels=dopc_pressure)
plot_peak_profile(redazo_I, redazo_Qxy, redazo_Qz, 'Redazo', savepath=plot_path / "redazo", labels=redazo_pressure)


C:\Users\ctyja\workspace\own\xrd-anlyutils\src\xrd_anlyutils\math\roi.py:59: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(crop, axis=axis)
